In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
df_train = pd.read_csv('dataset/train_prepared.csv')
df_train['waktu_setempat'] = pd.to_datetime(df_train['waktu_setempat'])
df_train = df_train.rename(columns={'waktu_setempat': 'timestamp', 'new_id': 'item_id'})

df_test = pd.read_csv('dataset/test_prepared.csv')
df_test['waktu_setempat'] = pd.to_datetime(df_test['waktu_setempat'])
df_test = df_test.rename(columns={'waktu_setempat': 'timestamp', 'new_id': 'item_id'})

## Null Handling and Skipping Timestamp Handling

### Null Handling

In [11]:
#Cek Null
print("df train\n", df_train.isna().sum(), "\n")
print("df test\n",df_test.isna().sum())

df train
 timestamp                0
rerata_kecepatan         0
lanes               244723
maxspeed                 0
highway                  0
item_id                  0
dtype: int64 

df test
 id               0
timestamp        0
lanes        78438
maxspeed         0
highway          0
item_id          0
dtype: int64


Dapat terlihat bahwa nilai null hanya terdapat pada feature lanes. Feature lanes merupakan feature yang menjelaskan jumlah lanes pada jalan tersebut. Berdasarkan intuisi dan domain knowledge, dapat diasumsikan bahwa apabila suatu lanes bernilai null, maka itu disebabkan karena hanya ada satu lanes pada jalan tersebut. Maka dari itu, nilai null pada feature tersebut akan diisi dengan nilai 1 

In [12]:
#fillna lanes
df_train['lanes'] = df_train['lanes'].fillna(1)
df_test['lanes'] = df_test['lanes'].fillna(1)

### Skipping Timestamp Handling

Pada kasus kali ini, digunakan metode filling data dari library autoML yaitu autogluon untuk handling skipping data (tanggal atau jam yang terlewat). Proses ini hanya perlu dilakukan pada train data

In [13]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [15]:
train_data = TimeSeriesDataFrame.from_data_frame(
    df_train,
    id_column="item_id",
    timestamp_column="timestamp"
)

In [16]:
train_data = train_data.to_regular_index(freq="H")
print(f"Data has frequency '{train_data.freq}'")

Data has frequency 'H'


In [17]:
train_data = train_data.fill_missing_values(method="auto")

In [18]:
# Turn back into pandas dataframe

df_train = pd.DataFrame(train_data)
df_train = df_train.reset_index()

In [19]:
df_train.head()

,item_id,timestamp,rerata_kecepatan,lanes,maxspeed,highway
0,691007296_21390008_1425033102,2020-02-01 01:00:00,29.126,3.0,30.0,trunk
1,691007296_21390008_1425033102,2020-02-01 02:00:00,34.772,3.0,30.0,trunk
2,691007296_21390008_1425033102,2020-02-01 03:00:00,34.161,3.0,30.0,trunk
3,691007296_21390008_1425033102,2020-02-01 04:00:00,39.383,3.0,30.0,trunk
4,691007296_21390008_1425033102,2020-02-01 05:00:00,42.153,3.0,30.0,trunk
